# 3C454.3 analysis notebook
## Prerequisities: in order to run this notebook you need the AGILE data.

In [ ]:
from agilepy.api.AGAnalysis import AGAnalysis

In [ ]:
%matplotlib notebook

## Creating a configuration file

In [ ]:
confFilePath = "./agilepy_conf.yaml"

In [ ]:
AGAnalysis.getConfiguration(
    confFilePath = confFilePath,
    userName = "bulgarelli",
    sourceName = "3C454d3b",
    tmin = 55500.0,
    tmax = 55520.0,
    timetype = "MJD",
    glon = 86.11,
    glat = -38.18,
    outputDir = "$HOME/agilepy_analysis",
    verboselvl = 1
)

## Obtaining the AGAnalysis object

In [ ]:
ag = AGAnalysis(confFilePath)

## Sources hypothesis

In [ ]:
sources = ag.loadSourcesFromCatalog("2AGL", rangeDist = (0, 5), show=True)

## Adding a source at runtime

In [ ]:
#newSourceDict = {
#    "glon" : 79.3,
#    "glat": 0.554,
#    "spectrumType" : "PowerLaw",
#    "flux": 1.69737e-07,
#    "index": 2.1
#}

#s = ag.addSource("CYGX3", newSourceDict)

#print(s)

## Deleting sources
Selection params = [name, dist, flux, sqrtts]

## Aggiungi un parametro print=False/True, False di default,  che se vale True stampa la lista delle  sorgenti come fa la riga successiva . 

In [ ]:
deletedSources = ag.deleteSources('flux <= 10e-08', show = True)

## Selecting sources

In [ ]:
sources = ag.selectSources("flux > 0", show = True)

## Free a source's parameter
Freeable params = [flux, index, index1, index2, cutoffEnergy, pivotEnergy, curvature]

In [ ]:
affectedSources = ag.freeSources('name == "2AGLJ2254+1609"', "flux", True, show=True)

In this example, only one source is affected.

## Analysis

In [ ]:
maplistfile = ag.generateMaps()

In [ ]:
! cat $maplistfile

In [ ]:
ag.displayCtsSkyMaps(maplistFile='/home/rt/agilepy_analysis/bulgarelli_3C454d3b_20200318-113534/3C454d3b_product.maplist4', smooth=3, catalogRegions="2AGL", catalogRegionsColor="green")

In [ ]:
ag.displayExpSkyMaps()

In [ ]:
ag.displayGasSkyMaps()

In [ ]:
ag.setOptions(galcoeff=[0.7])

In [ ]:
! cat $maplistfile

In [ ]:
ag.printOptions("model")

In [ ]:
#gal,iso,maplist = ag.calcBkg("2AGLJ2254+1609", galcoeff = [0.7], pastTimeWindow = 14)

In [ ]:
# print(gal)

In [ ]:
! cat $maplistfile

In [ ]:
ag.printOptions("model")

In [ ]:
ag.mle()

In [ ]:
ag.selectSources("sqrtTS > 0", show=True)

In [ ]:
ag.printOptions("model")

In [ ]:
! cat $maplistfile

In [ ]:
ag.mle()

In [ ]:
ag.selectSources("sqrtTS > 0")

### Light curve with default tmin and tmax

In [ ]:
lightCurveData = ag.lightCurve("2AGLJ2254+1609", binsize=86400)

In [ ]:
cat $lightCurveData

In [ ]:
print(lightCurveData)

In [ ]:
ag.displayLightCurve()

### Light curve with explicit tmin and tmax

In [ ]:
lightCurveData = ag.lightCurve("2AGLJ2254+1609", tmin=55510.0, tmax=55515.0, timetype="MJD", binsize=86400)

In [ ]:
cat $lightCurveData

In [ ]:
ag.displayLightCurve()

## Cleaning up

In [ ]:
ag.deleteAnalysisDir()